In [1]:
import pygame as pg
from random import sample

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
switch={
    #pg.K_RETURN:'RETURN',
    pg.K_ESCAPE:'ESCAPE',       #Exit the program
    #pg.K_t:'T',
    #pg.K_c:'C',
    #pg.K_UP:'UP',               #Used for cell selection
    #pg.K_DOWN:'DOWN',           #Used for cell selection
    #pg.K_LEFT:'LEFT',           #Used for cell selection
    #pg.K_RIGHT:'RIGHT',         #Used for cell selection
    #pg.K_o:'O',
    #pg.K_p:'P',
    #pg.K_l:'L',
    #pg.K_k:'K',
    #pg.K_j:'J',
    'MOUSE_PRESS':'MOUSE_PRESS' #Used to select a cell and indicate that an action should be performed
}       
        
def keypress(key):
    """ Given a pressed key, check if we care whether this key has been pressed. """
    return switch.get(key, "SP_NONE")

def getKey():
    """ Looks for the states of a set of keys and whether the QUIT key has been pressed. """
    key = None
    for event in pg.event.get():                   #For all pygame events
        if event.type == pg.QUIT:                  #If the user is trying to quit
            key = pg.K_ESCAPE                      #Handle as an escape key
        elif event.type == pg.KEYDOWN:             #If the user is pressing a key
            key = event.key                        #Get the key
        elif event.type == pg.MOUSEBUTTONDOWN:     #If the user has used a mouse button
            mouse_presses = pg.mouse.get_pressed() #Get what they've pressed
            if mouse_presses[0]:                   #If they've pressed left mouse
                key = 'MOUSE_PRESS'                #Set the key as a mousepress
    return keypress(key)                           #Return the result of the switch statement

In [3]:
class UserInterface:
    def __init__(self):
        #pg.display.init()
        self.cell_r_img = pg.image.load("img/cellr.png")
        self.cell_b_img = pg.image.load("img/cellb.png")
        self.cell_x_img = pg.image.load("img/cellx.png")
        self.cell_n_img = pg.image.load("img/celln.png")
        self.cell_u_img = pg.image.load("img/cellu.png")
        self.bg_img   = pg.image.load("img/bg.png")
        self.cell_size = (256, 128)
        
        self.font = pg.font.SysFont("calibri", 36)
        
        self.cells_x = 5
        self.cells_y = 6
        
        self.chest_start_pos = 1
        
        self.screen = pg.display.set_mode((self.cells_x*self.cell_size[0], self.cells_y*self.cell_size[1]))
    def update_screen(self, CellStates):
        self.update_background()
        self.update_cells(CellStates, self.cell_size[1]*self.chest_start_pos)
        
        pg.display.update()
    def update_cells(self, CellStates, y_offset=0):
        r_img = pg.transform.scale(self.cell_r_img, self.cell_size)
        b_img = pg.transform.scale(self.cell_b_img, self.cell_size)
        x_img = pg.transform.scale(self.cell_x_img, self.cell_size)
        n_img = pg.transform.scale(self.cell_n_img, self.cell_size)
        u_img = pg.transform.scale(self.cell_u_img, self.cell_size)
        for x in range(0, 5):
            for y in range(0, 5):
                cell_state = CellStates.get_cell_state((x, y))
                pos = (self.cell_size[0]*x, self.cell_size[1]*y + y_offset)
                word_type = cell_state.word_type
                known = cell_state.known
                
                #Display cell card
                if word_type == "R" and known:
                    self.screen.blit(r_img, pos)
                elif word_type == "B" and known:
                    self.screen.blit(b_img, pos)
                elif word_type == "X" and known:
                    self.screen.blit(x_img, pos)
                elif word_type == "N" and known:
                    self.screen.blit(n_img, pos)
                else:
                    self.screen.blit(u_img, pos)
                
                #Display cell word
                word = self.font.render(cell_state.word, 1, (0,0,0))                
                self.screen.blit(word, (pos[0] + int(self.cell_size[0]/3),
                                        pos[1] + int(self.cell_size[1]/3)))
    def update_background(self):
        bg_img = pg.transform.scale(self.bg_img, self.cell_size)
        for x in range(0, self.cells_x):
            for y in range(0, self.cells_y):
                pos = (self.cell_size[0]*x, self.cell_size[1]*y)
                #Display cell background
                self.screen.blit(bg_img, pos)
    
    def get_mouse_cell_pos(self, mouse_real_pos):
        """ Returns the cell that has been clicked in the form of an integer"""
        return (mouse_real_pos[0]//self.cell_size[0], mouse_real_pos[1]//self.cell_size[1])
                
                
    def __del__(self):
        pg.display.quit()
        

In [39]:
class CellState:
    def __init__(self, word="NOT SET", word_type="N"):
        self.word = word
        self.word_type = word_type
        self.known = False

class CellStateHandler:
    def __init__(self, wordlist, grid_size):
        #Generate word cards by randomly selecting words from the list, then choose their state
        selected_words = sample(wordlist, k=25)
        assigned_words = sample(selected_words, k=25)
        red_words = assigned_words[0:9]
        blue_words = assigned_words[9:17]
        fail_words = [assigned_words[17]]
        
        #Dictionary of word vectors
        self.vector = dict()
        
        self.x_size, self.y_size = grid_size
        
        self.cell_states = []
        self.red_unk, self.blu_unk, self.neu_unk, self.fai_unk = ([],[],[],[])
        for x in range(0, self.x_size):
            cell_state_arr = []
            for y in range(0, self.y_size):
                index_1D = x + y*self.x_size
                #Set the cell state (Red, Blue, Fail, or Neutral)
                if selected_words[index_1D] in red_words:
                    word_type = "R"
                    self.red_unk.append(selected_words[index_1D])
                elif selected_words[index_1D] in blue_words:
                    word_type = "B"
                    self.blu_unk.append(selected_words[index_1D])
                elif selected_words[index_1D] in fail_words:
                    word_type = "X"
                    self.fai_unk.append(selected_words[index_1D])
                else:
                    word_type = "N"
                    self.neu_unk.append(selected_words[index_1D])
                cell_state_arr.append(CellState(selected_words[index_1D], word_type))
                
                #Add the word and it's vector to the dictionary. NOT YET IMPLEMENTED.
                print("Word vector not yet implemented.")
                self.vector[selected_words[index_1D]] = ["NOT YET IMPLEMENTED"]
            self.cell_states.append(cell_state_arr)
    def get_cell_state(self, pos):
        """ Returns all information pertaining to the current cell """
        return self.cell_states[pos[0]][pos[1]]
    def set_flag_known(self, pos):
        """ Sets the known flag of the given cell as true. Returns true if the state of the cell was already known. """
        
        #If the word isn't already set to known
        if not self.cell_states[pos[0]][pos[1]].known:
            #Set the word to known
            self.cell_states[pos[0]][pos[1]].known = True
            
            #Remove the word from the corresponding unknown word list
            word, word_type = (self.cell_states[pos[0]][pos[1]].word, self.cell_states[pos[0]][pos[1]].word_type)
            if word_type == "R":
                self.red_unk.remove(word)
            elif word_type == "B":
                self.blu_unk.remove(word)
            elif word_type == "X":
                self.fai_unk.remove(word)
            elif word_type == "N":
                self.neu_unk.remove(word)
    def get_all_unknown_words(self):
        return (self.red_unk, self.blu_unk, self.neu_unk, self.fai_unk)
    def get_vector_given_key(self, word):
        """ Returns a word vector given the key. If the key does not exist, return none. """
        return self.vector.get(word, None)

In [40]:
def get_wordlist():
    wordlist = []
    with open("wordlist.txt", 'r') as wordfile:
        lines = wordfile.readlines()
        for line in lines:
            wordlist.append(line[0:-1])
    return wordlist

In [42]:
def main(UI):
    #Get cell state information based on the wordlist
    CellStates = CellStateHandler(get_wordlist(), (5,5))
    unknown_words = CellStates.get_all_unknown_words()
    
    #Set the user's selected key as none
    key = "SP_INITIAL"
    mouse_real_pos = pg.mouse.get_pos() #Get the current location of the mouse 

    #While the user isn't trying to quit
    done = False
    while not done:
        UI.update_screen(CellStates)
        
        """ User Input Operations """
        #If the user is trying to quit, exit the program
        if key in ("ESCAPE"):
            done = True
        #If the user has clicked, get their mouse position and perform selection operations
        elif key in ("MOUSE_PRESS"):
            i, j = UI.get_mouse_cell_pos(mouse_real_pos)
            y_offset = UI.chest_start_pos
            
            #If we're selecting a chest
            if j >= y_offset:
                #Set the cell to known
                CellStates.set_flag_known((i,j - y_offset))
                #Get the current unknown words (may have changed or been unchanged)
                unknown_words = CellStates.get_all_unknown_words()
                
                """ Testing - output word vectors """
                for word_list in unknown_words:
                    for word in word_list:
                        print(word, CellStates.get_vector_given_key(word))

        key = getKey() #Get any relevant keys being pressed
        mouse_real_pos = pg.mouse.get_pos() #Get the current location of the mouse
    

try:
    pg.init()
    UI = UserInterface()
    main(UI)
finally:
    del UI
    pg.quit()

Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
Word vector not yet implemented.
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT YET IMPLEMENTED']
['NOT Y

In [33]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = dict()
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

vectors = load_vectors("wiki-news-300d-1M.vec")

KeyboardInterrupt: 

In [15]:
v = vectors.copy()

print(123123)

NameError: name 'vectors' is not defined

In [14]:
print(list(v["work"]))

[]
